In [1]:
import pandas as pd
import numpy as np
from itertools import combinations,product
from sklearn.preprocessing import LabelEncoder

In [2]:
data = pd.read_csv("../../data/process_data.csv")
# data.info()

In [3]:
features = {}  # 创建空字典，将创建的特征放入字典中

# 组合特征

所有分类特征进行组合：

医院编码、出院诊断病种名称、出院诊断LENTH_MAX、顺序号、是否挂号、交易时间（新特征）、就诊的月数

In [4]:
## 交易时间

# 确认所有相关列是否在数据集中
if all(col in data.columns for col in ['交易时间DD_NN', '交易时间YYYY_NN', '交易时间YYYYMM_NN']):
    # 将日期组件合并成年月日的形式
    combined_date_feature = data['交易时间YYYY_NN'].astype(str) + '-' + data['交易时间YYYYMM_NN'].astype(str) + '-' + data['交易时间DD_NN'].astype(str)

    # 对合并后的日期进行标签编码
    label_encoder = LabelEncoder()
    # 对合并后的日期进行标签编码，不进行排序
    encoded_date_feature = label_encoder.fit_transform(combined_date_feature)

    # 将编码后的日期特征添加到字典中
    features['交易时间_NN'] = encoded_date_feature
else:
    # 如果列不存在，则使用NaN作为占位符
    features['交易时间_NN'] = pd.NA  # 使用pandas的NA值作为缺失值的占位符

# 显示字典中'交易时间'特征的前5个编码值
features['交易时间_NN'][:5]  # 显示前5个值作为示例

array([263, 232, 266, 251, 255])

In [5]:
def create_custom_feature_combinations(data, feature_lists):
    """
    自动为给定的特征列表创建自定义组合。
    :param data: Pandas DataFrame，包含原始数据
    :param feature_lists: 包含特征组合列表的列表
    :return: 更新后的特征字典
    """
    label_encoders = {}
    
    for feature_list in feature_lists:
        # 创建组合特征的名称
        combined_feature_name = '_and_'.join(feature_list)
        
        # 生成组合特征的字符串表示
        combined_feature = data[feature_list[0]].astype(str)
        for feature in feature_list[1:]:
            combined_feature += "_" + data[feature].astype(str)
        
        # 初始化标签编码器并对组合特征进行编码
        label_encoder = LabelEncoder()
        encoded_combined_feature = label_encoder.fit_transform(combined_feature)
        
        # 将编码后的组合特征存储到features字典中
        features[combined_feature_name] = encoded_combined_feature
        label_encoders[combined_feature_name] = label_encoder

    return features


In [6]:
data['交易时间_NN'] = features['交易时间_NN']

#创建多个自定义组合特征
feature_combinations = [
    ['医院编码_NN', '出院诊断病种名称_NN', '顺序号_NN'],
    # ['出院诊断LENTH_MAX', '顺序号_NN', '是否挂号','交易时间_NN'],  # 可能过拟合
    ['医院编码_NN','顺序号_NN'],
    ['医院编码_NN','出院诊断病种名称_NN'],
    ['医院编码_NN','是否挂号'],
    ['出院诊断LENTH_MAX','是否挂号'],
    ['出院诊断病种名称_NN','是否挂号'],
    ['交易时间_NN','就诊的月数'],
    # ['交易时间_NN','就诊的月数','顺序号_NN'],
    # ['出院诊断病种名称_NN','顺序号_NN'],
    # ['出院诊断LENTH_MAX','顺序号_NN'],
    # ['交易时间_NN','顺序号_NN'],
]

features = create_custom_feature_combinations(data, feature_combinations)
list(features.keys())

['交易时间_NN',
 '医院编码_NN_and_出院诊断病种名称_NN_and_顺序号_NN',
 '医院编码_NN_and_顺序号_NN',
 '医院编码_NN_and_出院诊断病种名称_NN',
 '医院编码_NN_and_是否挂号',
 '出院诊断LENTH_MAX_and_是否挂号',
 '出院诊断病种名称_NN_and_是否挂号',
 '交易时间_NN_and_就诊的月数']

# 比例特征

In [7]:
# 计算“就诊欺诈系数”
fraud_counts_hospital = data[data['RES'] == 1].groupby('医院编码_NN').size()
total_counts_hospital = data.groupby('医院编码_NN').size()
hospital_fraud_ratio = fraud_counts_hospital / total_counts_hospital
hospital_fraud_ratio = hospital_fraud_ratio.fillna(0)
features['就诊欺诈系数'] = data['医院编码_NN'].map(hospital_fraud_ratio)

# 计算“诊断欺诈系数”
fraud_counts_diagnosis = data[data['RES'] == 1].groupby('出院诊断病种名称_NN').size()
total_counts_diagnosis = data.groupby('出院诊断病种名称_NN').size()
diagnosis_fraud_ratio = fraud_counts_diagnosis / total_counts_diagnosis
diagnosis_fraud_ratio = diagnosis_fraud_ratio.fillna(0)
features['诊断欺诈系数'] = data['出院诊断病种名称_NN'].map(diagnosis_fraud_ratio)

feature_combination = [['就诊欺诈系数','诊断欺诈系数']]
list(features.keys())

['交易时间_NN',
 '医院编码_NN_and_出院诊断病种名称_NN_and_顺序号_NN',
 '医院编码_NN_and_顺序号_NN',
 '医院编码_NN_and_出院诊断病种名称_NN',
 '医院编码_NN_and_是否挂号',
 '出院诊断LENTH_MAX_and_是否挂号',
 '出院诊断病种名称_NN_and_是否挂号',
 '交易时间_NN_and_就诊的月数',
 '就诊欺诈系数',
 '诊断欺诈系数']

In [8]:
def calculate_proportions(data):

    # 定义要计算比例的列
    feature_1_cols = ['药品费发生金额_SUM', '检查费发生金额_SUM', '治疗费发生金额_SUM', '手术费发生金额_SUM', '床位费发生金额_SUM', '其它发生金额_SUM']
    feature_2_cols = ['药品费自费金额_SUM', '检查费自费金额_SUM', '治疗费自费金额_SUM', '手术费自费金额_SUM']
    feature_3_cols = ['药品费申报金额_SUM', '查费申报金额_SUM', '治疗费申报金额_SUM', '手术费申报金额_SUM', '床位费申报金额_SUM', '其它申报金额_SUM']

    # 计算比例
    for col in feature_1_cols:
        occurrence_col = col
        self_payment_col = col.replace('发生金额', '自费金额')
        claim_amount_col = col.replace('发生金额', '申报金额')

        # 自费金额比例
        if self_payment_col in data.columns:
            feature_key = f'{col}_自费比例'
            features[feature_key] = data.apply(
                lambda row: row[self_payment_col] / row[occurrence_col] if row[occurrence_col] != 0 else 0, 
                axis=1
            )
        
        # 申报金额比例
        if claim_amount_col in data.columns:
            feature_key = f'{col}_申报比例'
            features[feature_key] = data.apply(
                lambda row: row[claim_amount_col] / row[occurrence_col] if row[occurrence_col] != 0 else 0, 
                axis=1
            )

    return features

# 执行计算
features = calculate_proportions(data)

list(features.keys())


['交易时间_NN',
 '医院编码_NN_and_出院诊断病种名称_NN_and_顺序号_NN',
 '医院编码_NN_and_顺序号_NN',
 '医院编码_NN_and_出院诊断病种名称_NN',
 '医院编码_NN_and_是否挂号',
 '出院诊断LENTH_MAX_and_是否挂号',
 '出院诊断病种名称_NN_and_是否挂号',
 '交易时间_NN_and_就诊的月数',
 '就诊欺诈系数',
 '诊断欺诈系数',
 '药品费发生金额_SUM_自费比例',
 '药品费发生金额_SUM_申报比例',
 '检查费发生金额_SUM_自费比例',
 '检查费发生金额_SUM_申报比例',
 '治疗费发生金额_SUM_自费比例',
 '治疗费发生金额_SUM_申报比例',
 '手术费发生金额_SUM_自费比例',
 '手术费发生金额_SUM_申报比例',
 '床位费发生金额_SUM_申报比例',
 '其它发生金额_SUM_申报比例']

In [9]:
def create_fluctuation_features(data):

    selected_columns = data.columns[26:70].tolist()  # 替换为具体的列名范围
    visit_count_col = '就诊次数_SUM'  # 替换为就诊次数的列名

    for col in selected_columns:
        new_feature_name = f"{col}_波动"
        features[new_feature_name] = data[col] / data[visit_count_col]

    return features

# 执行构建特征的函数
features = create_fluctuation_features(data)

# 打印构建的特征字典中的一些示例
for key in list(features.keys())[-10:]:
    print(f"{key}: \n{features[key].head()}\n")

公务员医疗补助基金支付金额_SUM_波动: 
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64

城乡救助补助金额_SUM_波动: 
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64

基本个人账户支付_SUM_波动: 
0     6.729412
1    13.300000
2     5.200000
3     3.188696
4     2.461538
dtype: float64

非账户支付金额_SUM_波动: 
0    51.240882
1    78.138000
2    53.404444
3    56.509565
4    57.909615
dtype: float64

本次审批金额_SUM_波动: 
0    498.295294
1    727.716667
2    396.445111
3    596.444348
4    563.158846
dtype: float64

补助审批金额_SUM_波动: 
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64

医疗救助医院申请_SUM_波动: 
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64

残疾军人补助_SUM_波动: 
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64

民政救助补助_SUM_波动: 
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64

城乡优抚补助_SUM_波动: 
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
dtype: float64


In [10]:
len(list(features.keys()))

64

In [11]:
list(features.keys())

['交易时间_NN',
 '医院编码_NN_and_出院诊断病种名称_NN_and_顺序号_NN',
 '医院编码_NN_and_顺序号_NN',
 '医院编码_NN_and_出院诊断病种名称_NN',
 '医院编码_NN_and_是否挂号',
 '出院诊断LENTH_MAX_and_是否挂号',
 '出院诊断病种名称_NN_and_是否挂号',
 '交易时间_NN_and_就诊的月数',
 '就诊欺诈系数',
 '诊断欺诈系数',
 '药品费发生金额_SUM_自费比例',
 '药品费发生金额_SUM_申报比例',
 '检查费发生金额_SUM_自费比例',
 '检查费发生金额_SUM_申报比例',
 '治疗费发生金额_SUM_自费比例',
 '治疗费发生金额_SUM_申报比例',
 '手术费发生金额_SUM_自费比例',
 '手术费发生金额_SUM_申报比例',
 '床位费发生金额_SUM_申报比例',
 '其它发生金额_SUM_申报比例',
 '个人账户金额_SUM_波动',
 '统筹支付金额_SUM_波动',
 'ALL_SUM_波动',
 '可用账户报销金额_SUM_波动',
 '药品费发生金额_SUM_波动',
 '药品费自费金额_SUM_波动',
 '药品费申报金额_SUM_波动',
 '贵重药品发生金额_SUM_波动',
 '中成药费发生金额_SUM_波动',
 '中草药费发生金额_SUM_波动',
 '检查费发生金额_SUM_波动',
 '检查费自费金额_SUM_波动',
 '检查费申报金额_SUM_波动',
 '贵重检查费金额_SUM_波动',
 '治疗费发生金额_SUM_波动',
 '治疗费自费金额_SUM_波动',
 '治疗费申报金额_SUM_波动',
 '手术费发生金额_SUM_波动',
 '手术费自费金额_SUM_波动',
 '手术费申报金额_SUM_波动',
 '床位费发生金额_SUM_波动',
 '床位费申报金额_SUM_波动',
 '医用材料发生金额_SUM_波动',
 '高价材料发生金额_SUM_波动',
 '医用材料费自费金额_SUM_波动',
 '成分输血申报金额_SUM_波动',
 '其它发生金额_SUM_波动',
 '其它申报金额_SUM_波动',
 '一次性医用材料申报金额_SUM_波动',
 '起付线标准金额_MAX_波动

# 基本统计特征

## 差值特征

In [12]:
# 计算特征差值
features['月就诊天数差'] = data['月就诊天数_MAX'] - data['月就诊天数_AVG']
features['月就诊医院数差'] = data['月就诊医院数_MAX'] - data['月就诊医院数_AVG']
features['月统筹金额差'] = data['月统筹金额_MAX'] - data['月统筹金额_AVG']
features['月药品金额差'] = data['月药品金额_MAX'] - data['月药品金额_AVG']
features['医院_就诊天数差'] = data['医院_就诊天数_MAX'] - data['医院_就诊天数_AVG']
features['医院_统筹金差'] = data['医院_统筹金_MAX'] - data['医院_统筹金_AVG']
features['医院_药品差'] = data['医院_药品_MAX'] - data['医院_药品_AVG']

list(features.keys())

['交易时间_NN',
 '医院编码_NN_and_出院诊断病种名称_NN_and_顺序号_NN',
 '医院编码_NN_and_顺序号_NN',
 '医院编码_NN_and_出院诊断病种名称_NN',
 '医院编码_NN_and_是否挂号',
 '出院诊断LENTH_MAX_and_是否挂号',
 '出院诊断病种名称_NN_and_是否挂号',
 '交易时间_NN_and_就诊的月数',
 '就诊欺诈系数',
 '诊断欺诈系数',
 '药品费发生金额_SUM_自费比例',
 '药品费发生金额_SUM_申报比例',
 '检查费发生金额_SUM_自费比例',
 '检查费发生金额_SUM_申报比例',
 '治疗费发生金额_SUM_自费比例',
 '治疗费发生金额_SUM_申报比例',
 '手术费发生金额_SUM_自费比例',
 '手术费发生金额_SUM_申报比例',
 '床位费发生金额_SUM_申报比例',
 '其它发生金额_SUM_申报比例',
 '个人账户金额_SUM_波动',
 '统筹支付金额_SUM_波动',
 'ALL_SUM_波动',
 '可用账户报销金额_SUM_波动',
 '药品费发生金额_SUM_波动',
 '药品费自费金额_SUM_波动',
 '药品费申报金额_SUM_波动',
 '贵重药品发生金额_SUM_波动',
 '中成药费发生金额_SUM_波动',
 '中草药费发生金额_SUM_波动',
 '检查费发生金额_SUM_波动',
 '检查费自费金额_SUM_波动',
 '检查费申报金额_SUM_波动',
 '贵重检查费金额_SUM_波动',
 '治疗费发生金额_SUM_波动',
 '治疗费自费金额_SUM_波动',
 '治疗费申报金额_SUM_波动',
 '手术费发生金额_SUM_波动',
 '手术费自费金额_SUM_波动',
 '手术费申报金额_SUM_波动',
 '床位费发生金额_SUM_波动',
 '床位费申报金额_SUM_波动',
 '医用材料发生金额_SUM_波动',
 '高价材料发生金额_SUM_波动',
 '医用材料费自费金额_SUM_波动',
 '成分输血申报金额_SUM_波动',
 '其它发生金额_SUM_波动',
 '其它申报金额_SUM_波动',
 '一次性医用材料申报金额_SUM_波动',
 '起付线标准金额_MAX_波动

## 标准差特征

In [13]:
# specified_features = ['月就诊天数_MAX', '月就诊天数_AVG', '月就诊医院数_MAX', '月就诊医院数_AVG', 
#                       '月统筹金额_MAX', '月统筹金额_AVG', '月药品金额_MAX','月药品金额_AVG','医院_就诊天数_MAX','医院_就诊天数_AVG',
#                       '医院_统筹金_MAX','医院_统筹金_AVG','医院_药品_MAX','医院_药品_AVG']


specified_features = data.columns[4:20].tolist() + data.columns[26:68].tolist()

class_std = data[specified_features].std()

# 调整指定特征值
for column in specified_features:
    adjusted_column_name = f'{column}_调整'
    features[adjusted_column_name] = data[column] - class_std[column]

list(features.keys())

['交易时间_NN',
 '医院编码_NN_and_出院诊断病种名称_NN_and_顺序号_NN',
 '医院编码_NN_and_顺序号_NN',
 '医院编码_NN_and_出院诊断病种名称_NN',
 '医院编码_NN_and_是否挂号',
 '出院诊断LENTH_MAX_and_是否挂号',
 '出院诊断病种名称_NN_and_是否挂号',
 '交易时间_NN_and_就诊的月数',
 '就诊欺诈系数',
 '诊断欺诈系数',
 '药品费发生金额_SUM_自费比例',
 '药品费发生金额_SUM_申报比例',
 '检查费发生金额_SUM_自费比例',
 '检查费发生金额_SUM_申报比例',
 '治疗费发生金额_SUM_自费比例',
 '治疗费发生金额_SUM_申报比例',
 '手术费发生金额_SUM_自费比例',
 '手术费发生金额_SUM_申报比例',
 '床位费发生金额_SUM_申报比例',
 '其它发生金额_SUM_申报比例',
 '个人账户金额_SUM_波动',
 '统筹支付金额_SUM_波动',
 'ALL_SUM_波动',
 '可用账户报销金额_SUM_波动',
 '药品费发生金额_SUM_波动',
 '药品费自费金额_SUM_波动',
 '药品费申报金额_SUM_波动',
 '贵重药品发生金额_SUM_波动',
 '中成药费发生金额_SUM_波动',
 '中草药费发生金额_SUM_波动',
 '检查费发生金额_SUM_波动',
 '检查费自费金额_SUM_波动',
 '检查费申报金额_SUM_波动',
 '贵重检查费金额_SUM_波动',
 '治疗费发生金额_SUM_波动',
 '治疗费自费金额_SUM_波动',
 '治疗费申报金额_SUM_波动',
 '手术费发生金额_SUM_波动',
 '手术费自费金额_SUM_波动',
 '手术费申报金额_SUM_波动',
 '床位费发生金额_SUM_波动',
 '床位费申报金额_SUM_波动',
 '医用材料发生金额_SUM_波动',
 '高价材料发生金额_SUM_波动',
 '医用材料费自费金额_SUM_波动',
 '成分输血申报金额_SUM_波动',
 '其它发生金额_SUM_波动',
 '其它申报金额_SUM_波动',
 '一次性医用材料申报金额_SUM_波动',
 '起付线标准金额_MAX_波动

In [14]:
specified_features_std =data.columns[4:20].tolist() + data.columns[26:68].tolist() + data.columns[1:2].tolist()

feature_std_1 = data.groupby('出院诊断病种名称_NN')[specified_features_std].std()
feature_std_2 = data.groupby('医院编码_NN')[specified_features_std].std()

In [15]:
for column in specified_features_std:
    # 获取当前列的病种平均费用
    std_costs_by_disease = data['出院诊断病种名称_NN'].map(feature_std_1[column])
    
    # 计算差值
    adjusted_column_name_std = f'{column}_病种费用标准差值'
    features[adjusted_column_name_std] = data[column] - std_costs_by_disease


for column in specified_features_std:
    # 获取当前列的病种平均费用
    std_costs_by_hos = data['医院编码_NN'].map(feature_std_2[column])
    
    # 计算差值
    adjusted_column_name_std_2 = f'{column}_医院费用标准差值'
    features[adjusted_column_name_std_2] = data[column] - std_costs_by_hos


list(features.keys())


['交易时间_NN',
 '医院编码_NN_and_出院诊断病种名称_NN_and_顺序号_NN',
 '医院编码_NN_and_顺序号_NN',
 '医院编码_NN_and_出院诊断病种名称_NN',
 '医院编码_NN_and_是否挂号',
 '出院诊断LENTH_MAX_and_是否挂号',
 '出院诊断病种名称_NN_and_是否挂号',
 '交易时间_NN_and_就诊的月数',
 '就诊欺诈系数',
 '诊断欺诈系数',
 '药品费发生金额_SUM_自费比例',
 '药品费发生金额_SUM_申报比例',
 '检查费发生金额_SUM_自费比例',
 '检查费发生金额_SUM_申报比例',
 '治疗费发生金额_SUM_自费比例',
 '治疗费发生金额_SUM_申报比例',
 '手术费发生金额_SUM_自费比例',
 '手术费发生金额_SUM_申报比例',
 '床位费发生金额_SUM_申报比例',
 '其它发生金额_SUM_申报比例',
 '个人账户金额_SUM_波动',
 '统筹支付金额_SUM_波动',
 'ALL_SUM_波动',
 '可用账户报销金额_SUM_波动',
 '药品费发生金额_SUM_波动',
 '药品费自费金额_SUM_波动',
 '药品费申报金额_SUM_波动',
 '贵重药品发生金额_SUM_波动',
 '中成药费发生金额_SUM_波动',
 '中草药费发生金额_SUM_波动',
 '检查费发生金额_SUM_波动',
 '检查费自费金额_SUM_波动',
 '检查费申报金额_SUM_波动',
 '贵重检查费金额_SUM_波动',
 '治疗费发生金额_SUM_波动',
 '治疗费自费金额_SUM_波动',
 '治疗费申报金额_SUM_波动',
 '手术费发生金额_SUM_波动',
 '手术费自费金额_SUM_波动',
 '手术费申报金额_SUM_波动',
 '床位费发生金额_SUM_波动',
 '床位费申报金额_SUM_波动',
 '医用材料发生金额_SUM_波动',
 '高价材料发生金额_SUM_波动',
 '医用材料费自费金额_SUM_波动',
 '成分输血申报金额_SUM_波动',
 '其它发生金额_SUM_波动',
 '其它申报金额_SUM_波动',
 '一次性医用材料申报金额_SUM_波动',
 '起付线标准金额_MAX_波动

## 平均差值特征

In [16]:
## 计算类别0的平均值
specified_features_mean =data.columns[26:70].tolist()
# specified_features

# 计算类别0指定特征的平均值
class_mean = data[specified_features_mean].mean()

# 调整指定特征值
for column in specified_features_mean:
    adjusted_column_name_mean = f'{column}_AVG'
    features[adjusted_column_name_mean] = data[column] - class_mean[column]

list(features.keys())

['交易时间_NN',
 '医院编码_NN_and_出院诊断病种名称_NN_and_顺序号_NN',
 '医院编码_NN_and_顺序号_NN',
 '医院编码_NN_and_出院诊断病种名称_NN',
 '医院编码_NN_and_是否挂号',
 '出院诊断LENTH_MAX_and_是否挂号',
 '出院诊断病种名称_NN_and_是否挂号',
 '交易时间_NN_and_就诊的月数',
 '就诊欺诈系数',
 '诊断欺诈系数',
 '药品费发生金额_SUM_自费比例',
 '药品费发生金额_SUM_申报比例',
 '检查费发生金额_SUM_自费比例',
 '检查费发生金额_SUM_申报比例',
 '治疗费发生金额_SUM_自费比例',
 '治疗费发生金额_SUM_申报比例',
 '手术费发生金额_SUM_自费比例',
 '手术费发生金额_SUM_申报比例',
 '床位费发生金额_SUM_申报比例',
 '其它发生金额_SUM_申报比例',
 '个人账户金额_SUM_波动',
 '统筹支付金额_SUM_波动',
 'ALL_SUM_波动',
 '可用账户报销金额_SUM_波动',
 '药品费发生金额_SUM_波动',
 '药品费自费金额_SUM_波动',
 '药品费申报金额_SUM_波动',
 '贵重药品发生金额_SUM_波动',
 '中成药费发生金额_SUM_波动',
 '中草药费发生金额_SUM_波动',
 '检查费发生金额_SUM_波动',
 '检查费自费金额_SUM_波动',
 '检查费申报金额_SUM_波动',
 '贵重检查费金额_SUM_波动',
 '治疗费发生金额_SUM_波动',
 '治疗费自费金额_SUM_波动',
 '治疗费申报金额_SUM_波动',
 '手术费发生金额_SUM_波动',
 '手术费自费金额_SUM_波动',
 '手术费申报金额_SUM_波动',
 '床位费发生金额_SUM_波动',
 '床位费申报金额_SUM_波动',
 '医用材料发生金额_SUM_波动',
 '高价材料发生金额_SUM_波动',
 '医用材料费自费金额_SUM_波动',
 '成分输血申报金额_SUM_波动',
 '其它发生金额_SUM_波动',
 '其它申报金额_SUM_波动',
 '一次性医用材料申报金额_SUM_波动',
 '起付线标准金额_MAX_波动

In [17]:
len(list(features.keys()))

291

In [18]:
specified_features_mean =data.columns[4:20].tolist() + data.columns[26:68].tolist()+ data.columns[1:2].tolist()

feature_mean_1 = data.groupby('出院诊断病种名称_NN')[specified_features_mean].mean()
feature_mean_2 = data.groupby('医院编码_NN')[specified_features_mean].mean()

In [19]:
feature_mean_1.iloc[:-20]

,月就诊天数_AVG,月就诊医院数_MAX,月就诊医院数_AVG,就诊次数_SUM,月就诊次数_MAX,月就诊次数_AVG,月统筹金额_MAX,月统筹金额_AVG,月药品金额_MAX,月药品金额_AVG,...,基本统筹基金支付金额_SUM,公务员医疗补助基金支付金额_SUM,城乡救助补助金额_SUM,基本个人账户支付_SUM,非账户支付金额_SUM,本次审批金额_SUM,补助审批金额_SUM,医疗救助医院申请_SUM,残疾军人补助_SUM,一天去两家医院的天数
出院诊断病种名称_NN,,,,,,,,,,,,,,,,,,,,,
0,3.336150,1.574648,1.364038,14.090141,4.850704,3.488310,4585.979296,2833.975183,2738.437972,1857.430335,...,10254.123972,15.380338,131.542930,77.620648,2040.215859,12304.364761,213.221803,266.300197,0.000000,0.425352
1,2.993333,1.580000,1.294333,16.860000,4.460000,3.065333,5019.501600,2643.668523,3138.953600,1987.359737,...,12623.958800,0.000000,647.578800,171.579000,2585.449000,15044.789200,895.971800,24.238400,0.000000,0.260000
2,3.432099,1.504592,1.226100,20.270379,4.956946,3.485888,3446.742916,2265.271452,3092.360321,2092.381106,...,12566.172807,61.530293,238.041045,141.278594,1874.647991,14577.846751,351.991780,163.585419,1.970545,0.133180
3,4.365276,1.702860,1.380300,26.311682,6.253514,4.483959,3618.685313,2504.684659,3300.811842,2298.651302,...,14041.214998,64.967475,286.098177,139.387004,2036.207300,16342.137232,452.003558,152.422971,13.331997,0.395540
4,4.911846,1.900857,1.577538,30.275065,6.934029,5.123108,3471.451562,2442.026547,3158.860574,2243.733931,...,13877.969601,49.666258,229.221275,128.768874,1858.313511,16058.987372,358.916299,200.345330,5.508703,0.845322
5,5.439466,2.055077,1.710060,33.854627,7.636193,5.714263,3520.257718,2549.552806,3232.984414,2341.296182,...,14464.269378,43.228415,294.863571,121.239183,1856.337546,16677.564312,450.629918,166.858160,0.000000,1.145373
6,6.187081,2.217787,1.847103,39.039414,8.699343,6.579847,3678.984533,2658.123632,3476.956392,2531.866707,...,15145.947377,33.315083,297.530101,128.126241,1980.458201,17570.951289,457.341167,270.458262,0.000000,1.753916
7,6.900601,2.364691,1.951546,43.765464,9.737113,7.418159,3868.910129,2825.499707,3655.754278,2708.383967,...,15854.211778,61.177494,362.529858,122.489427,2089.666675,18577.131353,563.489916,383.167075,0.000000,2.414948
8,7.545833,2.493902,2.075271,48.242886,10.827236,8.180911,4323.846524,3086.525349,4055.911331,2966.845300,...,17323.614472,67.033943,380.625193,122.693140,2421.205193,20531.246697,599.551199,558.357144,12.376565,2.932927


In [20]:
# 创建一个新的DataFrame用于存放调整后的特征值
# adjusted_features = pd.DataFrame(index=data.index)

# 计算每个病例费用与其病种平均费用之间的差值
for column in specified_features_mean:
    # 获取当前列的病种平均费用
    mean_costs_by_disease = data['出院诊断病种名称_NN'].map(feature_mean_1[column])
    
    # 计算差值
    adjusted_column_name_mean = f'{column}_病种平均费用差'
    features[adjusted_column_name_mean] = data[column] - mean_costs_by_disease


list(features.keys())

['交易时间_NN',
 '医院编码_NN_and_出院诊断病种名称_NN_and_顺序号_NN',
 '医院编码_NN_and_顺序号_NN',
 '医院编码_NN_and_出院诊断病种名称_NN',
 '医院编码_NN_and_是否挂号',
 '出院诊断LENTH_MAX_and_是否挂号',
 '出院诊断病种名称_NN_and_是否挂号',
 '交易时间_NN_and_就诊的月数',
 '就诊欺诈系数',
 '诊断欺诈系数',
 '药品费发生金额_SUM_自费比例',
 '药品费发生金额_SUM_申报比例',
 '检查费发生金额_SUM_自费比例',
 '检查费发生金额_SUM_申报比例',
 '治疗费发生金额_SUM_自费比例',
 '治疗费发生金额_SUM_申报比例',
 '手术费发生金额_SUM_自费比例',
 '手术费发生金额_SUM_申报比例',
 '床位费发生金额_SUM_申报比例',
 '其它发生金额_SUM_申报比例',
 '个人账户金额_SUM_波动',
 '统筹支付金额_SUM_波动',
 'ALL_SUM_波动',
 '可用账户报销金额_SUM_波动',
 '药品费发生金额_SUM_波动',
 '药品费自费金额_SUM_波动',
 '药品费申报金额_SUM_波动',
 '贵重药品发生金额_SUM_波动',
 '中成药费发生金额_SUM_波动',
 '中草药费发生金额_SUM_波动',
 '检查费发生金额_SUM_波动',
 '检查费自费金额_SUM_波动',
 '检查费申报金额_SUM_波动',
 '贵重检查费金额_SUM_波动',
 '治疗费发生金额_SUM_波动',
 '治疗费自费金额_SUM_波动',
 '治疗费申报金额_SUM_波动',
 '手术费发生金额_SUM_波动',
 '手术费自费金额_SUM_波动',
 '手术费申报金额_SUM_波动',
 '床位费发生金额_SUM_波动',
 '床位费申报金额_SUM_波动',
 '医用材料发生金额_SUM_波动',
 '高价材料发生金额_SUM_波动',
 '医用材料费自费金额_SUM_波动',
 '成分输血申报金额_SUM_波动',
 '其它发生金额_SUM_波动',
 '其它申报金额_SUM_波动',
 '一次性医用材料申报金额_SUM_波动',
 '起付线标准金额_MAX_波动

In [21]:
# 调整指定特征值
# 计算每个病例费用与其病种平均费用之间的差值
for column in specified_features_mean:
    # 获取当前列的病种平均费用
    mean_costs = data['医院编码_NN'].map(feature_mean_2[column])
    
    # 计算差值
    adjusted_column_name_mean = f'{column}_医院平均费用差'
    features[adjusted_column_name_mean] = data[column] - mean_costs


list(features.keys())

['交易时间_NN',
 '医院编码_NN_and_出院诊断病种名称_NN_and_顺序号_NN',
 '医院编码_NN_and_顺序号_NN',
 '医院编码_NN_and_出院诊断病种名称_NN',
 '医院编码_NN_and_是否挂号',
 '出院诊断LENTH_MAX_and_是否挂号',
 '出院诊断病种名称_NN_and_是否挂号',
 '交易时间_NN_and_就诊的月数',
 '就诊欺诈系数',
 '诊断欺诈系数',
 '药品费发生金额_SUM_自费比例',
 '药品费发生金额_SUM_申报比例',
 '检查费发生金额_SUM_自费比例',
 '检查费发生金额_SUM_申报比例',
 '治疗费发生金额_SUM_自费比例',
 '治疗费发生金额_SUM_申报比例',
 '手术费发生金额_SUM_自费比例',
 '手术费发生金额_SUM_申报比例',
 '床位费发生金额_SUM_申报比例',
 '其它发生金额_SUM_申报比例',
 '个人账户金额_SUM_波动',
 '统筹支付金额_SUM_波动',
 'ALL_SUM_波动',
 '可用账户报销金额_SUM_波动',
 '药品费发生金额_SUM_波动',
 '药品费自费金额_SUM_波动',
 '药品费申报金额_SUM_波动',
 '贵重药品发生金额_SUM_波动',
 '中成药费发生金额_SUM_波动',
 '中草药费发生金额_SUM_波动',
 '检查费发生金额_SUM_波动',
 '检查费自费金额_SUM_波动',
 '检查费申报金额_SUM_波动',
 '贵重检查费金额_SUM_波动',
 '治疗费发生金额_SUM_波动',
 '治疗费自费金额_SUM_波动',
 '治疗费申报金额_SUM_波动',
 '手术费发生金额_SUM_波动',
 '手术费自费金额_SUM_波动',
 '手术费申报金额_SUM_波动',
 '床位费发生金额_SUM_波动',
 '床位费申报金额_SUM_波动',
 '医用材料发生金额_SUM_波动',
 '高价材料发生金额_SUM_波动',
 '医用材料费自费金额_SUM_波动',
 '成分输血申报金额_SUM_波动',
 '其它发生金额_SUM_波动',
 '其它申报金额_SUM_波动',
 '一次性医用材料申报金额_SUM_波动',
 '起付线标准金额_MAX_波动

In [22]:
len(list(features.keys()))

409

In [23]:
# def create_sum_feature(data):
#     # 定义要合并的列
#     sum_columns = [
#         '民政救助补助_SUM', '城乡优抚补助_SUM', '残疾军人补助_SUM', 
#         '医疗救助医院申请_SUM', '城乡救助补助金额_SUM', '公务员医疗补助基金支付金额_SUM'
#     ]

#     # 确保所有列都存在于数据中
#     for col in sum_columns:
#         if col not in data.columns:
#             raise ValueError(f"Column {col} not found in the data.")

#     # 计算总和
#     features['救助_补助金总和'] = {'救助_补助金总和': data[sum_columns].sum(axis=1)}

#     return features


# # 执行构建特征的函数
# create_sum_feature(data)

# 打印构建的特征的前几个值来进行检查
# print("救助_补助金总和: \n", features['救助_补助金总和'].head(20))

In [24]:
len(list(features.keys()))

409

In [25]:
list(features.keys())

['交易时间_NN',
 '医院编码_NN_and_出院诊断病种名称_NN_and_顺序号_NN',
 '医院编码_NN_and_顺序号_NN',
 '医院编码_NN_and_出院诊断病种名称_NN',
 '医院编码_NN_and_是否挂号',
 '出院诊断LENTH_MAX_and_是否挂号',
 '出院诊断病种名称_NN_and_是否挂号',
 '交易时间_NN_and_就诊的月数',
 '就诊欺诈系数',
 '诊断欺诈系数',
 '药品费发生金额_SUM_自费比例',
 '药品费发生金额_SUM_申报比例',
 '检查费发生金额_SUM_自费比例',
 '检查费发生金额_SUM_申报比例',
 '治疗费发生金额_SUM_自费比例',
 '治疗费发生金额_SUM_申报比例',
 '手术费发生金额_SUM_自费比例',
 '手术费发生金额_SUM_申报比例',
 '床位费发生金额_SUM_申报比例',
 '其它发生金额_SUM_申报比例',
 '个人账户金额_SUM_波动',
 '统筹支付金额_SUM_波动',
 'ALL_SUM_波动',
 '可用账户报销金额_SUM_波动',
 '药品费发生金额_SUM_波动',
 '药品费自费金额_SUM_波动',
 '药品费申报金额_SUM_波动',
 '贵重药品发生金额_SUM_波动',
 '中成药费发生金额_SUM_波动',
 '中草药费发生金额_SUM_波动',
 '检查费发生金额_SUM_波动',
 '检查费自费金额_SUM_波动',
 '检查费申报金额_SUM_波动',
 '贵重检查费金额_SUM_波动',
 '治疗费发生金额_SUM_波动',
 '治疗费自费金额_SUM_波动',
 '治疗费申报金额_SUM_波动',
 '手术费发生金额_SUM_波动',
 '手术费自费金额_SUM_波动',
 '手术费申报金额_SUM_波动',
 '床位费发生金额_SUM_波动',
 '床位费申报金额_SUM_波动',
 '医用材料发生金额_SUM_波动',
 '高价材料发生金额_SUM_波动',
 '医用材料费自费金额_SUM_波动',
 '成分输血申报金额_SUM_波动',
 '其它发生金额_SUM_波动',
 '其它申报金额_SUM_波动',
 '一次性医用材料申报金额_SUM_波动',
 '起付线标准金额_MAX_波动

# 分箱特征

**注意：** 
- 对于基于树的模型（如随机森林、梯度提升机等），不是很需要对数变换和分箱，因为这些模型能很好地处理非线性关系和异常值。但对于线性模型或某些基于距离的模型，这些转换可能有助于改善性能。

- 在本项目（医疗保险欺诈）中，进行对数变换不是很需要，因为异常值和偏差分布揭示了正常情况和欺诈情况的不同。

- 本次只进行十分位数分箱，用于构建逻辑回归模型

In [26]:
specified_features = data.columns[4:20].tolist() + data.columns[26:70].tolist()

# 十分位数分档的分档数
num_bins = 10

# 用于存储分选特征的空字典
feature_Binned = {}

# 为每个指定特征应用基于量值的分选
for feature in specified_features:
    # 为每个指定特征应用基于量值的分选
    if feature in data.columns:
        # 应用基于定量的分选并存储在字典中
        binned_feature_name = f"{feature}_Binned"
        feature_Binned[binned_feature_name] = pd.qcut(data[feature], q=num_bins, labels=False, duplicates='drop')


list(feature_Binned.keys())

['月就诊天数_AVG_Binned',
 '月就诊医院数_MAX_Binned',
 '月就诊医院数_AVG_Binned',
 '就诊次数_SUM_Binned',
 '月就诊次数_MAX_Binned',
 '月就诊次数_AVG_Binned',
 '月统筹金额_MAX_Binned',
 '月统筹金额_AVG_Binned',
 '月药品金额_MAX_Binned',
 '月药品金额_AVG_Binned',
 '医院_就诊天数_MAX_Binned',
 '医院_就诊天数_AVG_Binned',
 '医院_统筹金_MAX_Binned',
 '医院_统筹金_AVG_Binned',
 '医院_药品_MAX_Binned',
 '医院_药品_AVG_Binned',
 '个人账户金额_SUM_Binned',
 '统筹支付金额_SUM_Binned',
 'ALL_SUM_Binned',
 '可用账户报销金额_SUM_Binned',
 '药品费发生金额_SUM_Binned',
 '药品费自费金额_SUM_Binned',
 '药品费申报金额_SUM_Binned',
 '贵重药品发生金额_SUM_Binned',
 '中成药费发生金额_SUM_Binned',
 '中草药费发生金额_SUM_Binned',
 '检查费发生金额_SUM_Binned',
 '检查费自费金额_SUM_Binned',
 '检查费申报金额_SUM_Binned',
 '贵重检查费金额_SUM_Binned',
 '治疗费发生金额_SUM_Binned',
 '治疗费自费金额_SUM_Binned',
 '治疗费申报金额_SUM_Binned',
 '手术费发生金额_SUM_Binned',
 '手术费自费金额_SUM_Binned',
 '手术费申报金额_SUM_Binned',
 '床位费发生金额_SUM_Binned',
 '床位费申报金额_SUM_Binned',
 '医用材料发生金额_SUM_Binned',
 '高价材料发生金额_SUM_Binned',
 '医用材料费自费金额_SUM_Binned',
 '成分输血申报金额_SUM_Binned',
 '其它发生金额_SUM_Binned',
 '其它申报金额_SUM_Binned',
 '一次性医用材料申报金

# 多项式特征

**注意：**

- 同分箱特征类似，对于基于树的模型（如随机森林、梯度提升机等）不是很需要多项式特征，因为它们能够很好的扑捉非线性关系。但对于线性模型或某些基于距离的模型，这些特征可能有助于改善性能。
- 本项目（医疗保险欺诈）中构建诊断、医院与费用特征多项交互特征，用于构建更好的逻辑回归模型



In [27]:
from sklearn.preprocessing import PolynomialFeatures

# 选择用于创建多项式特征的连续型特征
continuous_features = data.columns[4:20].tolist() + data.columns[26:70].tolist()

# 初始化多项式特征转换器，设置degree=2，不包含原始特征
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)

# 对所选特征应用多项式转换
poly_features = poly.fit_transform(data[continuous_features])

# 从转换后的特征中过滤掉原始特征
# 转换后的数据中的前N列是原始特征，N为连续型特征的数量，所以我们从N+1列开始选择
N = len(continuous_features)
poly_features_filtered = poly_features[:, N:]

# 获取新的特征名（跳过原始特征名）
feature_names = [f"{name}_多项式" for name in poly.get_feature_names_out(continuous_features)][N:]

# 创建空字典并填充多项式特征
poly_features_dict = {}
for i, feature_name in enumerate(feature_names):
    feature_Binned[feature_name] = poly_features_filtered[:, i].tolist()

# 查看字典的键（即特征名）
print(feature_Binned.keys())


dict_keys(['月就诊天数_AVG_Binned', '月就诊医院数_MAX_Binned', '月就诊医院数_AVG_Binned', '就诊次数_SUM_Binned', '月就诊次数_MAX_Binned', '月就诊次数_AVG_Binned', '月统筹金额_MAX_Binned', '月统筹金额_AVG_Binned', '月药品金额_MAX_Binned', '月药品金额_AVG_Binned', '医院_就诊天数_MAX_Binned', '医院_就诊天数_AVG_Binned', '医院_统筹金_MAX_Binned', '医院_统筹金_AVG_Binned', '医院_药品_MAX_Binned', '医院_药品_AVG_Binned', '个人账户金额_SUM_Binned', '统筹支付金额_SUM_Binned', 'ALL_SUM_Binned', '可用账户报销金额_SUM_Binned', '药品费发生金额_SUM_Binned', '药品费自费金额_SUM_Binned', '药品费申报金额_SUM_Binned', '贵重药品发生金额_SUM_Binned', '中成药费发生金额_SUM_Binned', '中草药费发生金额_SUM_Binned', '检查费发生金额_SUM_Binned', '检查费自费金额_SUM_Binned', '检查费申报金额_SUM_Binned', '贵重检查费金额_SUM_Binned', '治疗费发生金额_SUM_Binned', '治疗费自费金额_SUM_Binned', '治疗费申报金额_SUM_Binned', '手术费发生金额_SUM_Binned', '手术费自费金额_SUM_Binned', '手术费申报金额_SUM_Binned', '床位费发生金额_SUM_Binned', '床位费申报金额_SUM_Binned', '医用材料发生金额_SUM_Binned', '高价材料发生金额_SUM_Binned', '医用材料费自费金额_SUM_Binned', '成分输血申报金额_SUM_Binned', '其它发生金额_SUM_Binned', '其它申报金额_SUM_Binned', '一次性医用材料申报金额_SUM_Binned', '起付线标准金额_MAX_Binned

In [28]:
len(list(feature_Binned.keys()))

1890

In [29]:
data = data.drop(['交易时间_NN'],axis=1)

# 从字典创建新特征集DataFrame，并指定索引
new_features_data = pd.DataFrame(features, index=data.index)

# 将新特征集DataFrame与原始数据集合并
combined_data = data.join(new_features_data)

# 保存到新文件中
combined_data.to_csv('../../data/new_features.csv')

In [30]:
# 合并字典
merged_dict = {**features, **feature_Binned}

# 从字典创建新特征集DataFrame，并指定索引
new_features_data_2 = pd.DataFrame(merged_dict, index=data.index)

# 将新特征集DataFrame与原始数据集合并
combined_data = data.join(new_features_data_2)

# 保存到新文件中
combined_data.to_csv('../../data/new_features_data_2.csv')